In [28]:
import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql import types

findspark.init()

spark: SparkSession = SparkSession.builder \
    .appName("JupyterLocalSpark") \
    .master("local[*]") \
    .getOrCreate()

sc: SparkContext = spark.sparkContext

In [29]:
movies_schema = types.StructType([ \
    types.StructField("heroID", types.IntegerType(), nullable=False),
    types.StructField("heroName", types.StringType(), nullable=False),
])

df_names = spark.read.option("sep", ' ').schema(movies_schema).csv("data/Marvel+Names")
df_names.show(5)

+------+--------------------+
|heroID|            heroName|
+------+--------------------+
|     1|24-HOUR MAN/EMMANUEL|
|     2|3-D MAN/CHARLES CHAN|
|     3|    4-D MAN/MERCURIO|
|     4|             8-BALL/|
|     5|                   A|
+------+--------------------+
only showing top 5 rows



In [30]:
df_heros = spark.read.text("data/Marvel+Graph")
df_heros.show(5)

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
+--------------------+
only showing top 5 rows



In [31]:
df_heros = df_heros.select(
    func.explode(func.split("value", " ")).alias("heroID"),
    func.size(func.split("value", " ")).alias("herosCount")
).filter(func.col("heroID") != "")

df_heros.show(5)

+------+----------+
|heroID|herosCount|
+------+----------+
|  5988|        50|
|   748|        50|
|  1722|        50|
|  3752|        50|
|  4655|        50|
+------+----------+
only showing top 5 rows



In [32]:
df_heros = df_heros \
    .groupBy("heroID") \
    .sum("herosCount") \
    .select("heroID", func.col("sum(herosCount)").alias("relSum"))
    
df_heros.show(5)

+------+------+
|heroID|relSum|
+------+------+
|  2069| 66190|
|  3210| 38761|
|   829| 11360|
|  6194|  2997|
|  4821|  1447|
+------+------+
only showing top 5 rows



In [33]:
df_heros = df_heros \
    .join(df_names, on="heroID", how="left") \
    .select("heroName", "relSum") \
    .sort(func.desc("relSum"))
    
df_heros.show(10)

+--------------------+------+
|            heroName|relSum|
+--------------------+------+
|     CAPTAIN AMERICA|215651|
|SPIDER-MAN/PETER PAR|192498|
|IRON MAN/TONY STARK |186668|
|MR. FANTASTIC/REED R|184199|
|THING/BENJAMIN J. GR|183898|
|SCARLET WITCH/WANDA |175000|
|HUMAN TORCH/JOHNNY S|174822|
|BEAST/HENRY &HANK& P|174576|
|    WOLVERINE/LOGAN |173673|
|INVISIBLE WOMAN/SUE |172938|
+--------------------+------+
only showing top 10 rows

